# TP 5: Les procédures de chiffrements asymétriques

Dans la famille des procédures de chiffrements asymétriques, nous retrouvons beaucoup de procédure de chiffrement. Comme vu en cours, nous savons que chaque procédure de chiffrement asymétrique repose sur une foncton à sens unique. Nous nous attarderons dans ce TP sur trois d'entre elles:
- le cryptosystème RSA, basé sur la fonction à sens unique de l'exponentiation modulaire; 
- le cryptosystème de El Gamal, basé sur la fonction à sens unique qui fait le produit d'entiers premiers;
- la procédure d'échange de clef de Diffie-Hellman, basée également sur l'exponentiation modulaire.

Ce TP est composé de 3 parties :
1. Le chiffrement RSA
2. Le chiffrement El Gamal
3. La lecture de fichiers par blocs

In [ ]:
# Imports utiles
import random
import numpy as np
import matplotlib.pyplot as plt
import math
import time

rng = random.Random(0x123456789)

## Partie 1 - Le cryptosytème RSA

Ce cryptosystème est apparu dans les années 1978 à la suite de la découverte d'un résultat mêlant le théorème d'Euler au théorème de Sun Zi par Rivest, Shamir et Adleman. Pour plus détails, vous pouvez consulter [la page Wikipédia](https://fr.wikipedia.org/wiki/Chiffrement_RSA).

Pour manipuler efficacement les objets du système RSA, nous rappelons les fonctions que nous avons réalisés précédemment dans d'autres TP.

### 1.1. Rappels des TP précédents

Au TP précédent, on a vu qu'on pouvait calculer des inverses modulaires grâce aux coefficients de Bézout. Les fonctions ci-dessous implémentent cela.

In [ ]:
def bezout(a, b):
    """
    Donne deux entiers u et v tels que a*u + b*v = pgcd(a, b).

    Args: les deux entiers à partir desquels calculer la combinaison de bézout

    Outputs: les entiers u et v tels que a*u + b*v = pgcd(a, b)
    """
    if b == 0:
        return 1, 0
    q, r = divmod(a, b)

    x, y = bezout(b, r)
    return y, x - y * q

In [ ]:
def inv_mod(a, p):
    """
    Calcule l'inverse de a modulo p.

    Args: l'entier à inverser et le modulo

    Outputs: l'inverse de cet entier dans l'ensemble modulaire
    """
    u, v = bezout(a, p)

    pgcd = u * a + v * p
    if pgcd != 1:
        raise ValueError(f"{a} n'est pas inversible dans Z / {p}Z")

    return u % p

### 1.2. Programme qui effectue le chiffrement RSA

Pour les choix de nombre premier on peut regarder la [table](https://www.walter-fendt.de/html5/men/primenumbers_en.htm) qui recense tous les premiers de $1$ à un mille milliards.

On rappelle la procédure de chiffrement de RSA ci-dessous:
- on tire deux grands entiers premiers $p$ et $q$ (on ne fera pas attention aux recommandations de sécurité pour simplifier);
- on calcule $n=pq$;
- on calcule $\varphi(n)=(p-1)(q-1)$;
- on choisit un assez grand entier $e$ premier à $\varphi(n)$;
- on calcule son inverse $d$ dans $\mathbb{Z}/\varphi(n)\mathbb{Z}$;
- on publie la clé publique $(n,e)$;
- on conserve la clé privée $(n, d)$ (on n'a plus besoin de $p, q$ !).

*Remarque :* En pratique, l'inverse dans $\mathbb{Z}/\varphi(n)\mathbb{Z}^*$ est calculé dans $\mathbb{Z}/(p-1)\mathbb{Z}^*$ et dans $\mathbb{Z}/(q-1)\mathbb{Z}^*$, le message étant calculé grâce au théorème de Sun Zi (communément appelé théorème des restes chinois). Plus de détails sur [cette page](https://www.di-mgt.com.au/crt_rsa.html).

#### 1.2.a. Génération de clés


**TODO**: Ecrire une fonction qui à partir de deux entiers premiers $p$ et $q$ génère la clef publique et la clef privée RSA en tirant l'indice $e$ au hasard et assez grand ($>\log(n))$. Pour tester votre fonction prenez:
$$ 
p=1000133 \text{ et } q=1000151.
$$

In [ ]:
def RSA_keys(p, q):
    """
    Generate RSA public and private keys.
    Args:
        p (int): A prime number.
        q (int): Another prime number.
    Returns:
        tuple: A tuple containing the public key (n, e) and the private key (n, d).
            - pubkey (tuple): The public key, consisting of (n, e).
            - privkey (tuple): The private key, consisting of (n, d).
    """

    pass # TODO


# TODO: tester la fonction

#### 1.2.b. Chiffrement et déchiffrement

On considère un message $M$ (qui pour l'instant est un entier) et on souhaite le chiffrer avec en utilisant la méthode RSA. Nous devons alors calculer:
$$ 
C = M^e \mod n.
$$
En général, ce chiffrement RSA est seulement utilisé pour échanger des petits messages (le générateur de Diffie-Hellman), et donc on testera sur des nombres de taille confortable.

**TODO**: En utilisant la fonction `pow`, coder une fonction `RSA_cypher(M, n, e)` et `RSA_decypher(C, n, d)` qui respectivement chiffre et déchiffre les messages avec les clefs publiques et privées en entrée. Puis tester vos fonctions pour voir si vos clés sont correctes.

In [ ]:
def RSA_cypher(M, n, e):
    pass  # TODO


def RSA_decypher(C, n, d):
    pass  # TODO

In [ ]:
# Exemple avec des petites valeurs
M = 5
p, q = 13, 17

pubkey, privkey = RSA_keys(p, q)

print(pubkey, privkey)
print((pubkey[1] * privkey[1]) % ((p - 1) * (q - 1)))

C = RSA_cypher(M, pubkey[0], pubkey[1])
print("Le message chiffré est ", C)
N = RSA_decypher(C, privkey[0], privkey[1])
print("Le message déchiffré est ", N)

In [ ]:
# Exemple avec des grandes valeurs
M = 1376286482
p = 1000133
q = 1000151
pubkey, privkey = RSA_keys(p, q)
C = RSA_cypher(M, pubkey[0], pubkey[1])
print("Le message chiffré est ", C)

N = RSA_decypher(C, privkey[0], privkey[1])
print("Le message déchiffré est ", N)

## Partie 2 - Conversion de fichiers en entiers

Dans tous les encodages, le nombre de bits correspondant à un caractère est un multiple de 8: il peut être encodé sur 1 ou plusieurs octets. De ce fait, les fichiers sont organisés en octets, et donc en les lisant, on va les découper en bloc d'octets, d'une taille décidée *a priori*.

Le risque avec cette approche est d'arriver en fin de fichier et de n'avoir qu'un bloc plus petit que la taille prescrit : avec des blocs de taille $N$ octets, le fichier aura $q \times N + r$ octets pour un certain $0 \leq r < N$, et donc le dernier bloc aura une taille $N - r$. La fonction ci-dessous ajoute des caractères/octets nuls `\x00` pour compléter ce dernier bloc.

In [ ]:
def readblocks_null_padding(fname, blocksize):
    """
    Reads a file in blocks of a specified size, padding the last block with null bytes if necessary.

    Args:
        fname (str): The path to the file to be read.
        blocksize (int): The size of each block in bytes.

    Returns:
        list: A list of integers, each representing a block of bytes from the file.
    """
    file_blocks = []  # liste des entiers représentés par chaque bloc d'octets
    eof = False
    with open(fname, "rb") as fstream:  # r pour reading, b pour bytes
        while not eof:
            byte_block = fstream.read(blocksize)

            pad_len = blocksize - len(byte_block)
            if pad_len > 0:  # reached end of file
                byte_block = byte_block.ljust(blocksize, b"\x00")
                eof = True

            int_block = int.from_bytes(byte_block, byteorder="big")
            file_blocks.append(int_block)

    return file_blocks

### 2.1. Le padding (méthode PKCS7)

Cette approche qui rajoute des octets nuls est simple à implémenter mais est largement imparfaite : une fois le bloc rembourré, on ne peut pas savoir de de combien d'octets il a été complété. L'approche [PKCS #7](https://en.wikipedia.org/wiki/PKCS_7) règle ce problème en ajoutant $N - r$ octets de *valeur* $N - r$ en fin de bloc. Si $r = 0$, on crée un nouveau bloc de taille $N$ où chaque octet prend la valeur $N$. Cette méthode est très bien illustrée sur le site de [Node Security](https://node-security.com/posts/cryptography-pkcs-7-padding/).

*Remarque:* Quelle est la valeur maximale que peut prendre un octet ? Que peut-on en déduire sur la taille $N$ maximale des blocs qu'on peut prendre ?

**TODO**  
En utilisant la méthode `int.to_bytes`, compléter la fonction `pad_pkcs7` ci-dessous.

In [ ]:
def pad_pkcs7(block: bytearray, pad_len: int):
    """
    Pads the given bytearray using the PKCS#7 padding scheme.
    Args:
        block (bytearray): The bytearray to be padded.
        pad_len (int): The length of the padding to be added.
    Returns:
        bytearray: The padded bytearray.
    """
    pass  # TODO


# Tests
print(
    'Padding de b"\\x00" avec 3 blocs: b"\\x00\\x03\\x03\\x03" --',
    pad_pkcs7(bytearray(1), 3) == b"\x00\x03\x03\x03",
)
print(
    'Padding de b"" avec 4 blocs: b"\\x04\\x04\\x04\\x04" --',
    pad_pkcs7(bytearray(0), 4) == b"\x04\x04\x04\x04",
)

**TODO**  
Coder une méthode de lecture de fichier par blocs qui prend en compte ce padding. Tester également avec un fichier dont le nombre de blocs est divisible par la taille choisie.

In [ ]:
def readblocks(fname, blocksize):
    """
    Reads a file in blocks of a specified size, padding the last block using the PKCS#7 padding scheme.

    Args:
        fname (str): The path to the file to be read.
        blocksize (int): The size of each block in bytes.

    Returns:
        list: A list of integers, each representing a block of bytes from the file.
    """
    pass  # TODO


readblocks("test-padding.txt", 5)
# On reconnait 6 fois "Pigs " puis "Pigs\x01" (padding de 1)

# TODO un autre test

### 2.2. Unpadding

Pour chiffrer un fichier, on effectue les opérations suivantes, 
$$ M \overset{\text{pad}}{\longmapsto} \bar{M} \overset{\text{cipher}}{\longmapsto} C $$
avec $\bar{M}$ et $C$ qui sont des listes de nombres, dont le nombre d'octets correspondra à la taille des blocs choisie.

Ainsi en déchiffrant le fichier chiffré, on obtiendra le fichier $\bar{M}$ rembourré. Il faut encore supprimer le *padding* avant de le sauvegarder. Pour pouvoir sauvegarder le fichier chiffré, on ne voudra pas forcément retirer le rembourrage.

In [ ]:
def unpad_pkcs7(block: bytearray):
    """
    Remove PKCS#7 padding from a given bytearray.
    This function assumes that the input block is padded correctly according to the PKCS#7 standard.
    Args:
        block (bytearray): The padded bytearray from which the padding needs to be removed.
    Returns:
        bytearray: The bytearray with the PKCS#7 padding removed.
    Raises:
        ValueError: If the padding is incorrect.
    """
    pass  # TODO


# Tests
print(
    'Unpadding de b"\\x02\\x02" --',
    unpad_pkcs7(b"\x02\x02") == b"",
)

print(
    'Unpadding de b"\\x01\\x01" --',
    unpad_pkcs7(b"\x01\x01") == b"\x01",
)

print("Padding incorrect de b'\\x01\\x02' --", end=" ")
try:
    unpad_pkcs7(b"\x01\x02")
    print(False)
except:
    print(True)

In [ ]:
def writeblocks(fname, int_blocks, blocksize, unpad=True):
    """
    Writes a list of integer blocks to a binary file.
    Args:
        fname (str): The name of the file to write to.
        blocks (list of int): The list of integer blocks to write.
        blocksize (int): The size of each block in bytes.
        unpad (bool, optional): Whether to unpad the last block using PKCS#7 padding. Defaults to True.
    Returns:
        None
    """
    # TODO définir le vecteur de bytearray et prendre en compte l'unpadding
    byte_blocks = []

    with open(fname, "wb") as fstream:  # w pour writing, b pour bytes
        for bblock in byte_blocks:
            fstream.write(bblock)


blocksize = 5
blocks = readblocks(
    "test-padding.txt", blocksize
)  # suppose que `readblocks` marche bien :^)
writeblocks("test-unpadding.txt", blocks, blocksize, unpad=True)
# ^ vérifier qu'on retrouve bien le contenu original de `test-padding.txt`

## Partie 3 - Le cryptosystème de El Gamal

Ce cryptosystème est apparu en 1984 par Taher Elgamal, un cryptographe égyptien. Cette procédure de chiffrement est un peu plus complexe que RSA à mettre en place mais elle a le bénéfice d'avoir le même nveau de sécurité que RSA avec une clef deux fois plus petite. Si vous êtes curieux, vous pouvez consulter la [page wikipédia du chiffrement de El Gamal](https://fr.wikipedia.org/wiki/Cryptosyst%C3%A8me_de_ElGamal).

### 3.1. L'exponentiation modulaire

Nous codons ici de la fonction "exponentiation modulaire" détaillée en cours qui permet d'effectuer le calcul de $a^b$ en utilisant $O(\log_2(b))$ multiplications au lieu de $O(b)$ (ce qui représente un gain de temps exponentiel). 

*Exemple*: en utilisant une méthode d'exponentiation naïve:
$$
a^{13}=\underbrace{a\cdot a\cdot \ldots \cdot a}_{13 \text{ fois}}.
$$
En utilisant une méthode d'exponentiation rapide:
$$
a^{13}=a^{12}\cdot a=\left( a^6 \right)^2\cdot a= \left( \left(a^3\right)^2 \right)^2 \cdot a= \left(\left( a^2 \cdot a\right)^2\right)^2 \cdot a.
$$
La méthode naïve effectue $13$ multiplications tandis que la méthode rapide effectue seulement $5$ multiplications.

Dans cette partie, nous allons coder l'algorithme d'exponentiation naïve et rapide afin de comparer leurs performances à l'aide de la fonction *time* de la librairie *time*.

#### 3.1.a. Implémentations

**TODO**  
Coder les fonctions `exp_naive(x, a, n)` et `exp_rapide(x, a, n)` qui calcule $x^a \mod n$.

In [ ]:
def exp_naive(x, a, n):
    pass  # TODO


def exp_rapide(x, a, n):
    pass  # TODO


print(exp_naive(15, 14, 103), exp_rapide(15, 14, 103))
print(pow(15, 14, 103))

#### 3.1.b. Comparaison des temps de calcul

**TODO**  
A l'aide de la librairie *matplotlib*, tracer un graphe des temps pris par chacune de ces deux fonctions pour des exposants entre $10$ et $10^4$ en utilisant une échelle appropriée pour effectuer un calcul modulo $n=100000037\simeq 10^8$.

Pour la version naïve, calculer le temps moyen sur 20 itérations. Pour la version rapide, moyenner sur 100 itérations.

In [ ]:
x = 214267
n = 100000037
Interv = np.geomspace(10, 10**5, num=3000, dtype=int)

# TODO: comparer les temps d'exécution des deux fonctions

### 3.2. Le logarithme discret

Comme nous l'avons vu en cours, le logarithme discret est une réciproque partielle à la fonction d'exponentiation modulaire.
On considère $p$ un nombre premier et on considère $a$ un générateur du groupe multiplicatif $\mathbb{Z}/p\mathbb{Z}^*.$
Il s'agit de l'application:
$$
\log_a: x \mapsto c \text{ le plus petit entier positif réalisant } x^c = a \mod p .
$$

#### 3.2.a. Validation de générateur

**TODO**  
Coder une fonction `is_generator(x, p)` qui vérifie si $x$ est un générateur de $\mathbb{Z}/p\mathbb{Z}^*.$ Testez cette fonction.

In [ ]:
def is_generator(x, p):
    """
    Détermine si un nombre x est un générateur du groupe multiplicatif des entiers modulo p.
    Args:
        x (int): Le nombre à tester comme générateur.
        p (int): Le module du groupe multiplicatif.
    Returns:
        bool: True si x est un générateur du groupe multiplicatif modulo p, False sinon.
    """
    pass  # TODO


# TODO test

#### 3.2.b. Trouver un générateur

**TODO**  
Servez-vous de la fonction précédente pour concevoir un algorithme recherchant un générateur de $\mathbb{Z}/p\mathbb{Z}^*$ avec $p=1000151$.

In [ ]:
def find_generator(p):
    pass  # TODO


# TODO test

#### 3.2.c. Logarithme discret brute force

**TODO**  
Implémenter le logarithme discret avec une méthode de recherche naïve.

In [ ]:
def log_discret(x, a, p):
    """
    Cherche un exposant positif b le plus petit possible tel que a^b = x mod p
    """
    res = 1
    for i in range(1, p):
        res = (res * a) % p
        if res == x % p:
            return i


print(pow(7, 2, 11))
print(log_discret(5, 7, 11) == 2)

### 3.3. Chiffrement El Gamal

Pour cela on rappelle, la procédure de génération de clefs pour utiliser l'algorithme de El-Gamal:
- choisir $p$ un nombre premier et $g$ un générateur du groupe $\left( \mathbb{Z}/{p\mathbb{Z}}^*, \times, 1 \right)$; 
- choisir aléatoirement $s$ un nombre et calculer $\beta=g^s$; 
- publier la clé publique $(p,g,\beta)$;
- garder la clef secrète $s$.

Une fois ces clefs produites, le chiffré du message en clair $M$ est calculé en tirant aléatoirement un élément de ${\mathbb{Z}}/{(p-1)\mathbb{Z}}$ et en renvoyant:
$$
C(M)=(y_1,y_2)=\left( g^k \mod p,  M\times \beta^k \mod p \right).
$$
Puis, la procédure de déchiffrement appliqué à $(y_1,y_2)$ est donnée par:
$$
D(y_1,y_2)=y_2\times \left(y_1^s \right)^{-1}.
$$

**TODO**  
Coder une fonction `El_Gamal_keys(p,g)` qui renvoie la clef privée et la clef publique du chiffrement de El-Gamal, une fonction `El_Gamal_cypher(M,key_pub)` qui chiffre le message $M$ et une fonction `El_Gamal_decypher(C,key_priv)` qui déchiffre le message.

In [ ]:
def El_Gamal_keys(p, g):
    pass  # TODO


def El_Gamal_cypher(M, key_pub):
    pass  # TODO


def El_Gamal_decypher(C, key_priv):
    pass  # TODO

In [ ]:
# Test des fonctions
p = 21817
M = 3539
key_pub, key_priv = El_Gamal_keys(p, 2)
C = El_Gamal_cypher(M, key_pub)
D = El_Gamal_decypher(C, key_priv)
print("Le aclé publique est ", key_pub, " tandis que la clef privée est ", key_priv)
print("Le message en clair est ", M)
print("Le message chiffré est ", C, " puis le message déchiffré est ", D)

## Appendice - Théorème de Sun Zi

Le chiffrement RSA se base sur le fait qu'à partir d'un nombre $n = p \times q$ avec $p$ et $q$ premiers «grands» (aujourd'hui, cela signifie que $n$ est de 2048 bits), il est difficile de retrouver les facteurs premiers $p$, $q$, et donc la caractéristique d'Euler $\varphi(n) = (p - 1) \times (q - 1)$. 

**Théorème faible de Sun Zi:** Soit $p,q$ deux entiers premiers entre eux et notons $n = p q$. Alors:
$$ \mathbb{Z}/n\mathbb{Z}\simeq  \mathbb{Z}/p\mathbb{Z}\times\mathbb{Z}/q\mathbb{Z}.$$

On capture cet isomorphisme dans une structure `pq = ProductSpace(p, q)` qui permet de convertir des nombres entre l'espace $\mathbb{Z}/n\mathbb{Z}$ et l'espace produit $\mathbb{Z}/p\mathbb{Z} \times \mathbb{Z}/q\mathbb{Z}$, dans les deux sens.

La fonction `pq.to_prodspace(x)` effectue cette opération:
$$
x\mod n \rightarrow \begin{cases}
x&\mod p, \\
x&\mod q.
\end{cases}
$$

Tandis que la fonction `pq.from_prodspace(yp, yq)` est la réciproque de `pq.to_prodspace(x)` qui trouve l'unique $x \mod n$ tel que:
$$
\begin{cases} x = y_p\ [p], \\ x = y_q\ [q], \end{cases}
$$

In [ ]:
class ProductSpace:
    def __init__(self, p, q):
        self.p, self.q = p, q
        self.n = p * q

        # Calcul des solutions fondamentales xp, xq,
        # représentants de (1, 0) et de (0, 1) dans Z/nZ
        ap, aq = bezout(p, q)
        self.xp, self.xq = ap * q, aq * p

    def to_prodspace(self, x):
        """
        Associe à un entier ses classes de congruence modulo p et q.

        Args: l'entier

        Outputs: un couple d'entier modulo p et q
        """
        return x % self.p, x % self.q

    def from_prodspace(self, yp, yq):
        """
        Chercher la solution du système x = yp mod p, x = yq mod q.

        Args: les entiers à droite du système ci-dessus y_p,y_q

        Outputs: la solution modulo n du système
        """
        return (yp * self.xp + yq * self.xq) % self.n